In [ ]:
!pip install langchain-openai #Libraries Required

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import sqlite3
from datetime import datetime

# Initialize the local model via LM Studio
llm = ChatOpenAI(
    openai_api_key="dummy_key",
    base_url="http://localhost:1234/v1",
    model_name="local-model",
    temperature=0.9
)

# Database setup
def setup_database():
    conn = sqlite3.connect('language_learning.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS mistakes 
                 (user_id TEXT, language TEXT, mistake TEXT, correction TEXT, timestamp TEXT)''')
    conn.commit()
    conn.close()

class LanguageLearningBot:
    def __init__(self):
        self.user_id = "user_" + str(datetime.now().timestamp())
        self.user_name = ""
        self.known_language = ""
        self.learning_language = ""
        self.level = ""
        self.mistakes = []
        self.current_scene = ""
        self.conversation_history = []
        setup_database()

    def get_user_preferences(self):
        print("# Language Learning Chatbot\n")
        print("Hello! What is your name?\n")
        self.user_name = input("**User:** ")
        print(f"\nNice to meet you, {self.user_name}! What language do you want to learn?\n")
        self.learning_language = input("**User:** ")
        print("\nWhat is your native language?\n")
        self.known_language = input("**User:** ")
        print(f"\nWhat is your current level in {self.learning_language}? (Beginner, Intermediate, Advanced)\n")
        self.level = input("**User:** ")

    def save_mistake(self, mistake, correction):
        conn = sqlite3.connect('language_learning.db')
        c = conn.cursor()
        c.execute("INSERT INTO mistakes VALUES (?, ?, ?, ?, ?)",
                  (self.user_id, self.learning_language, mistake, correction, 
                   datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
        conn.commit()
        conn.close()
        self.mistakes.append({"mistake": mistake, "correction": correction})

    def generate_scene(self):
        scene_prompt = PromptTemplate(
            input_variables=["language", "level", "known_language"],
            template="You are a language teacher. The user is a {level} learner of {language}, and their native language is {known_language}. Generate a simple conversational scene in {language} (only in {language}, no other languages). The scene should be a short dialogue (1-2 sentences) in a specific setting (e.g., a café, a store, a park, a library) between two people. If the user is a beginner, include a translation in {known_language} in parentheses after each dialogue line; otherwise, do not include translations. Do not include an answer or example response in the question. End with a question in {known_language} asking the user how they would respond in {language} to continue the dialogue. Format the response as: 'Scene: [setting]\n[Person 1]: [sentence] ([translation if beginner])\n[Person 2]: [sentence] ([translation if beginner])\nQuestion: [question]'"
        )
        response = llm.invoke(scene_prompt.format(
            language=self.learning_language,
            level=self.level,
            known_language=self.known_language
        )).content
        if self.learning_language.lower() == "french" and not any(char in response.lower() for char in ["é", "è", "à", "le ", "la "]):
            print("Error: Scene not in correct language. Retrying...")
            response = llm.invoke(scene_prompt.format(
                language=self.learning_language,
                level=self.level,
                known_language=self.known_language
            )).content
        self.current_scene = response
        self.conversation_history.append(response)
        return response

    def continue_scene(self, user_response):
        continue_prompt = PromptTemplate(
            input_variables=["language", "level", "known_language", "previous_scene", "user_response"],
            template="You are a language teacher. The user is a {level} learner of {language}, and their native language is {known_language}. Here is the previous scene and conversation:\n{previous_scene}\nUser: {user_response}\nContinue the conversation by generating a 1-2 sentence response in {language} from Person 2, responding naturally to the user's input. If the user is a beginner, include a translation in {known_language} in parentheses after each dialogue line; otherwise, do not include translations. Do not include an answer or example response in the question. Then, ask a question in {known_language} about how the user would respond next in {language}. Format the response as: '[Person 2]: [sentence] ([translation if beginner])\nQuestion: [question]'"
        )
        history = "\n".join(self.conversation_history[-3:])
        response = llm.invoke(continue_prompt.format(
            language=self.learning_language,
            level=self.level,
            known_language=self.known_language,
            previous_scene=history,
            user_response=user_response
        )).content
        self.conversation_history.append(f"User: {user_response}")
        self.conversation_history.append(response)
        return response

    def provide_hints(self):
        hints_prompt = PromptTemplate(
            input_variables=["language", "level", "known_language", "scene"],
            template="You are a language teacher. The user is a {level} learner of {language}, and their native language is {known_language}. Based on this scene:\n{scene}\nProvide a list of 3-5 relevant {language} words or short phrases (with their meanings in {known_language}) that the user can use to construct a response. For beginners, focus on basic vocabulary. For intermediate learners, include expressions or variations that can make responses more natural. Do not provide a full response or example sentence. Format the response as: '- [word/phrase]: [meaning in known_language]'"
        )
        history = "\n".join(self.conversation_history[-3:])
        hints = llm.invoke(hints_prompt.format(
            language=self.learning_language,
            level=self.level,
            known_language=self.known_language,
            scene=history
        )).content
        return hints

    def check_response(self, user_input):
        check_prompt = PromptTemplate(
            input_variables=["language", "text", "known_language", "scene", "user_name"],
            template="You are a language teacher. The user, {user_name}, is learning {language} and their native language is {known_language}. Here is the current scene and conversation:\n{scene}\nThe user is responding as Person 1 to a question about what they would say in {language}. Check this {language} text: '{text}'. Be extremely strict: if there are any grammatical mistakes, politeness issues, typos, if the response doesn’t perfectly match the conversation context in every detail, or if it’s not entirely in {language}, provide a short correction (1-2 sentences) in {known_language} and the correct version in {language}. Format the response as: 'Good try, {user_name}! However, in {language}, it’s more polite/correct to say: \"[corrected text]\".' Only if the text is grammatically perfect, polite, matches the context precisely in every way, and is entirely in {language}, say 'Correct, {user_name}! To make your response more conversational, you could say: \"[suggestion]\".' in {known_language}. If the text is grammatically correct but doesn’t fully align with the context, say 'Good {language}, {user_name}, but that doesn’t quite fit the conversation perfectly. Instead, you could say: \"[suggestion]\".' in {known_language}. Ensure all suggestions and corrections are in {language}."
        )
        history = "\n".join(self.conversation_history[-3:])
        analysis = llm.invoke(check_prompt.format(
            language=self.learning_language,
            text=user_input,
            known_language=self.known_language,
            scene=history,
            user_name=self.user_name
        )).content
        if len(analysis) < 10 or "correct" not in analysis.lower() and "good try" not in analysis.lower():
            return f"Sorry, {self.user_name}, I couldn't analyze your response properly. Please try a simpler sentence in {self.learning_language}."
        return analysis

    def get_review(self):
        review_prompt = PromptTemplate(
            input_variables=["language", "mistakes", "known_language", "conversation", "user_name"],
            template="You are a language teacher. The user, {user_name}, is learning {language} and their native language is {known_language}. Here is the conversation:\n{conversation}\nThey made these mistakes: {mistakes}. If there are mistakes, write a short review (2-3 sentences) in {known_language} explaining what areas (e.g., grammar, vocabulary, politeness) the user should focus on to improve their {language} skills. If there are no mistakes, reference the conversation and suggest 1-2 specific areas for improvement (e.g., using more polite expressions, expanding vocabulary) to help them advance in {language}. Do not assume a response if the user quit before answering the last question."
        )
        if not self.mistakes:
            mistakes_str = "No mistakes were made."
        else:
            mistakes_str = "\n".join([f"Mistake: {m['mistake']} -> Correction: {m['correction']}" 
                                     for m in self.mistakes])
        conversation = "\n".join(self.conversation_history)
        review = llm.invoke(review_prompt.format(
            language=self.learning_language,
            mistakes=mistakes_str,
            known_language=self.known_language,
            conversation=conversation,
            user_name=self.user_name
        )).content
        if len(review) < 10:
            return f"In {self.known_language}, {self.user_name}: You did well in {self.learning_language}. Try using more polite expressions and expanding your vocabulary to improve."
        return review

    def run(self):
        self.get_user_preferences()
        print("\n## Practice Conversation\n")
        print(f"Great, {self.user_name}! Let’s practice a conversation.  \n")
        scene = self.generate_scene()
        print(f"**Scene:** {scene}\n")
        if self.level.lower() in ["beginner", "intermediate"]:
            print("### Hints\n")
            print(f"{self.provide_hints()}\n")
        while True:
            user_input = input("**User:** ")
            print("")
            if user_input.lower() == 'quit':
                break
            analysis = self.check_response(user_input)
            print("### Feedback\n")
            print(f"{analysis}\n")
            if "say:" in analysis.lower():
                mistake = user_input
                correction = analysis.split("say: ")[-1].strip().strip('."')
                self.save_mistake(mistake, correction)
                print("I’ll note this down for your review at the end.\n")
            continued_scene = self.continue_scene(user_input)
            print(f"**Scene (continued):** {continued_scene}\n")
            if self.level.lower() in ["beginner", "intermediate"]:
                print("### Hints\n")
                print(f"{self.provide_hints()}\n")
        print("## Session Review\n")
        print(f"{self.get_review()}\n")
        print("### Mistakes Made This Session\n")
        if not self.mistakes:
            print("No mistakes were recorded in this session.")
        else:
            for mistake in self.mistakes:
                print(f"- **Mistake:** {mistake['mistake']}  \n  **Correction:** {mistake['correction']}\n")

if __name__ == "__main__":
    bot = LanguageLearningBot()
    bot.run()